In [180]:
import pandas as pd
import numpy as np

In [181]:
game_df = pd.read_csv("raw_data/Twitch_game_data.csv")
global_df = pd.read_csv("raw_data/Twitch_global_data.csv")
streamers_df = pd.read_csv("raw_data/twitchdata-update.csv")

In [182]:
def dateToDateTime(dataframe):
    df = dataframe.copy()
    df["day"] = 1
    df = df.set_index(pd.to_datetime(df[["year", "Month", "day"]]))
    df = df.drop(["year", "Month", "day"], axis=1)
    return df

## Global viewers chart

In [183]:
Viewers = global_df[["year","Month","Avg_viewers"]]

In [184]:
Viewers = dateToDateTime(Viewers)

In [185]:
Viewers.to_csv("data/global_viewers.csv")

## Games views for ridgeplot

In [186]:

game_df.drop(game_df[game_df["Game"] == "Just Chatting"].index, inplace=True)

In [187]:
game_avg = game_df.copy()
game_avg.drop(columns=["Rank","Hours_watched","Hours_Streamed","Peak_viewers","Peak_channels","Streamers","Avg_channels","Avg_viewer_ratio"] ,inplace=True)

In [188]:
game_avg["day"] = 1
game_avg["date"] = pd.to_datetime(game_avg[["Year", "Month", "day"]])
game_avg = game_avg.drop(["Year", "Month", "day"], axis=1)

In [189]:
top_games = game_avg.groupby(['Game'])['Avg_viewers'].sum().sort_values(ascending=False)[:10].index.to_list()

In [190]:
games = []

for game in top_games:
    games.append(game_avg[game_avg["Game"] == game])

In [191]:
for g in games:
    g.rename(columns={"Avg_viewers" : g.iloc[0]["Game"]},inplace=True)
    g.drop(columns="Game", inplace=True)

e:\Technologie\Programming\twitch-to-the-moon\.venv\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
e:\Technologie\Programming\twitch-to-the-moon\.venv\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [192]:
from functools import reduce

pd.concat(games)

top_games_avg_viewers = reduce(lambda  left,right: pd.merge(left,right, on="date", how="outer"), games).fillna(0)

In [202]:
top_games_avg_viewers.to_csv("data/top_games_viewers.csv")

for i,game in enumerate(games):
    game.to_csv("data/"+ games[i]["Game"].iloc[0] +".csv")

## Average viewers of all games

In [194]:
all_games = game_avg.groupby(['Game'])['Avg_viewers'].sum().sort_values(ascending=False).index.to_list()

games = []

for game in all_games:
    games.append(game_avg[game_avg["Game"] == game])

In [195]:
for g in games:
    g.rename(columns={"Avg_viewers" : g.iloc[0]["Game"]},inplace=True)
    g.drop(columns="Game", inplace=True)


In [196]:
pd.concat(games)

all_games_avg_view = reduce(lambda  left,right: pd.merge(left,right, on="date", how="outer"), games).fillna(0)

In [200]:
all_games_avg_view.to_csv("data/all_games_viewers.csv")

## Domination of the Game